In [15]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD

In [16]:
from sample_data import *
users, ratings, books = get_saved_samples()

In [12]:
books['Book-Title'] = books['Book-Title'].str.lower()
books['Book-Author'] = books['Book-Author'].str.lower()
# Eliminamos posibles duplicados
books.drop_duplicates(subset='ISBN', keep='first', inplace=True)

In [ ]:
books['combined_features'] = books['Book-Title'] + ' ' + books['Book-Author']


In [13]:
# Vectorización de los títulos de los libros
tfidf_vectorizer = TfidfVectorizer(analyzer='word', stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(books['combined_features'])


In [14]:
# Utilizamos SVD para reducir la dimensionalidad
svd = TruncatedSVD(n_components=200)  # El número de componentes puede ser ajustado
tfidf_matrix_reduced = svd.fit_transform(tfidf_matrix)

# Cálculo de similitud del coseno en la matriz de dimensionalidad reducida
cosine_sim_matrix = cosine_similarity(tfidf_matrix_reduced, tfidf_matrix_reduced)


MemoryError: Unable to allocate 549. GiB for an array with shape (271360, 271360) and data type float64

In [ ]:
def book_recommendations(title, cosine_sim_matrix=cosine_sim_matrix, books=books):
    # Intentamos encontrar el libro
    try:
        idx = books.index[books['Book-Title'] == title.lower()].tolist()[0]
    except IndexError:
        print(f"El libro con título '{title}' no se encuentra en el dataset.")
        return []

    # Puntajes de similitud en orden descendente
    sim_scores = list(enumerate(cosine_sim_matrix[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Puntajes de los 10 libros más similares
    sim_scores = sim_scores[1:11]

    # Índices de libros
    book_indices = [i[0] for i in sim_scores]

    # Títulos de los 10 libros más similares
    return books['Book-Title'].iloc[book_indices]


In [ ]:
book_title = "classical mythology"
recommendations = book_recommendations(book_title)
print(recommendations)